In [2]:
import multiprocessing as mp  #parallel processing module
import time
import pandas as pd
import numpy as np

In [20]:
my_dict = {"Average": [200, 300],
    "Standard_deviation": [100, 300],
    "Exponential_average": [500,700]}

data = 'FP_python_challenge/data/data.csv' #dataset gotten from the girhub repo
df = pd.read_csv(data)

def rollingWindow(df):
   
    col = list(df.columns) # getting the list of columns from the dataframe
    df.set_index(col[0], inplace=True) #setting the first column as the index of the dataframe
    avg_windows = my_dict["Average"]  # getting the average window sizes from the dictionary
    std_windows = my_dict["Standard_deviation"]  # getting the standard deviation windows sizes from the dictionary
    exp_avg_windows = my_dict['Exponential_average']  # getting the Exponential_average windows sizes from the dictionary
    
    for i in range(len(col)):
        if i != 0: #ignoring the index
            for j in range(0, len(avg_windows)):
                df[f'Roll_avg_{avg_windows[j]}_{col[i]}'] = df[col[i]].rolling(avg_windows[j]).mean()#using the rolling function to get the average
                df[f'Roll_std_{std_windows[j]}_{col[i]}'] = df[col[i]].rolling(std_windows[j]).std() #using the rolling function to get the std
                df[f'Exp_avg_{exp_avg_windows[j]}_{col[i]}'] = df[col[i]].ewm(span=exp_avg_windows[j], adjust=False).mean() #using the rolling m=function to get the moving average
    
    df.reset_index(inplace=True) # setting back the index to default 
    
    return df 

In [21]:
t1 = time.time()
with mp.Pool(20) as pool:
    result = pool.imap(rollingWindow, data, chunksize=10000)
    updated_df = rollingWindow(df)  

t2 = time.time()
print(f'Time taken: {t2-t1} ms')

Time taken: 0.6228406429290771 ms


In [22]:
updated_df.head()

,readable_ts,A,B,C,D,E,Roll_avg_200_A,Roll_std_100_A,Exp_avg_500_A,Roll_avg_300_A,...,Exp_avg_500_D,Roll_avg_300_D,Roll_std_300_D,Exp_avg_700_D,Roll_avg_200_E,Roll_std_100_E,Exp_avg_500_E,Roll_avg_300_E,Roll_std_300_E,Exp_avg_700_E
0,2022-02-11 04:00:00,650.0,5.99882,650.051819,55.7047,532.294128,NaN,NaN,650.0,NaN,...,650.0,NaN,NaN,650.0,NaN,NaN,650.0,NaN,NaN,650.0
1,2022-02-11 04:00:01,650.0,5.99882,650.051819,55.7047,532.294128,NaN,NaN,650.0,NaN,...,650.0,NaN,NaN,650.0,NaN,NaN,650.0,NaN,NaN,650.0
2,2022-02-11 04:00:02,650.0,5.99882,650.051819,55.7047,532.294128,NaN,NaN,650.0,NaN,...,650.0,NaN,NaN,650.0,NaN,NaN,650.0,NaN,NaN,650.0
3,2022-02-11 04:00:03,650.0,5.99882,650.051819,55.7047,532.294128,NaN,NaN,650.0,NaN,...,650.0,NaN,NaN,650.0,NaN,NaN,650.0,NaN,NaN,650.0
4,2022-02-11 04:00:05,650.0,5.99882,650.051819,55.7047,532.294128,NaN,NaN,650.0,NaN,...,650.0,NaN,NaN,650.0,NaN,NaN,650.0,NaN,NaN,650.0


In [23]:
updated_df.tail()

,readable_ts,A,B,C,D,E,Roll_avg_200_A,Roll_std_100_A,Exp_avg_500_A,Roll_avg_300_A,...,Exp_avg_500_D,Roll_avg_300_D,Roll_std_300_D,Exp_avg_700_D,Roll_avg_200_E,Roll_std_100_E,Exp_avg_500_E,Roll_avg_300_E,Roll_std_300_E,Exp_avg_700_E
335595,2022-02-15 03:59:54,0.0,9.967005,9.871390e-15,1.342282,2138.288086,313.451527,214.184672,172.158017,208.967684,...,172.158017,1.765101,1.342243,136.671032,2126.828964,6.432105,172.158017,2132.544417,9.852061,136.671032
335596,2022-02-15 03:59:55,0.0,9.967005,9.871390e-15,1.342282,2138.288086,313.451527,216.173837,171.470760,208.967684,...,171.470760,1.758389,1.339293,136.281100,2126.799694,6.615426,171.470760,2132.528987,9.839377,136.281100
335597,2022-02-15 03:59:57,0.0,9.967005,9.871390e-15,1.342282,2138.288086,313.451527,218.029073,170.786246,208.967684,...,170.786246,1.751678,1.336303,135.892281,2126.770424,6.800201,170.786246,2132.513556,9.826653,135.892281
335598,2022-02-15 03:59:58,0.0,9.967005,9.871390e-15,1.342282,2138.288086,313.451527,219.753772,170.104464,208.967684,...,170.104464,1.744966,1.333273,135.504571,2126.741154,6.977305,170.104464,2132.498126,9.813888,135.504571
335599,2022-02-15 03:59:59,0.0,9.967005,9.871390e-15,1.342282,2138.288086,313.250743,221.350987,169.425405,208.967684,...,169.425405,1.738255,1.330202,135.117968,2126.755360,7.147310,169.425405,2132.482695,9.801082,135.117968


In [4]:
my_dict = {"Average": [200, 300],
    "Standard_deviation": [100, 300],
    "Exponential_average": [500,700]}

data = 'FP_python_challenge/data/data.csv' #dataset gotten from the girhub repo
df = pd.read_csv(data)

In [5]:
def rollingWindow(df):
    
    columns = list(df.columns)
    num_columns = len(columns)
    
    df.set_index(columns[0], inplace=True)
    
    avg_window = my_dict["Average"]
    std_window = my_dict["Standard_deviation"]
    exp_avg_window = my_dict["Exponential_average"]
    
    for i in range(num_columns):
        if i != 0:
            for j in range(0, len(avg_window)):
                df[f'Roll_avg_{avg_window[j]}_{columns[i]}'] = df[columns[i]].rolling(avg_window[j]).mean() 
                df[f'Roll_std_{std_window[j]}_{columns[i]}'] = df[columns[i]].rolling(std_window[j]).std()
                df[f'Roll_exp_avg_{exp_avg_window[j]}_{columns[i]}'] = df[columns[i]].ewm(span=exp_avg_window[j], adjust=False, ignore_na=True).mean()
                
    df.reset_index(inplace=True) # setting back the index to default 
    
    return df 

with mp.Pool(20) as pool:
    result = pool.imap(rollingWindow, data, chunksize=10000)
    updated_df = rollingWindow(df)  

In [6]:
updated_df.head()

,readable_ts,A,B,C,D,E,Roll_avg_200_A,Roll_std_100_A,Roll_exp_avg_500_A,Roll_avg_300_A,...,Roll_exp_avg_500_D,Roll_avg_300_D,Roll_std_300_D,Roll_exp_avg_700_D,Roll_avg_200_E,Roll_std_100_E,Roll_exp_avg_500_E,Roll_avg_300_E,Roll_std_300_E,Roll_exp_avg_700_E
0,2022-02-11 04:00:00,650.0,5.99882,650.051819,55.7047,532.294128,NaN,NaN,650.0,NaN,...,55.7047,NaN,NaN,55.7047,NaN,NaN,532.294128,NaN,NaN,532.294128
1,2022-02-11 04:00:01,650.0,5.99882,650.051819,55.7047,532.294128,NaN,NaN,650.0,NaN,...,55.7047,NaN,NaN,55.7047,NaN,NaN,532.294128,NaN,NaN,532.294128
2,2022-02-11 04:00:02,650.0,5.99882,650.051819,55.7047,532.294128,NaN,NaN,650.0,NaN,...,55.7047,NaN,NaN,55.7047,NaN,NaN,532.294128,NaN,NaN,532.294128
3,2022-02-11 04:00:03,650.0,5.99882,650.051819,55.7047,532.294128,NaN,NaN,650.0,NaN,...,55.7047,NaN,NaN,55.7047,NaN,NaN,532.294128,NaN,NaN,532.294128
4,2022-02-11 04:00:05,650.0,5.99882,650.051819,55.7047,532.294128,NaN,NaN,650.0,NaN,...,55.7047,NaN,NaN,55.7047,NaN,NaN,532.294128,NaN,NaN,532.294128


In [7]:
s = pd.Series([1, 2, 3, 4, 5])
s.rolling(2).mean()

0    NaN
1    1.5
2    2.5
3    3.5
4    4.5
dtype: float64